# Plan-and-Execute

This notebook shows how to create a "plan-and-execute" style agent. This is
heavily inspired by the [Plan-and-Solve](https://arxiv.org/abs/2305.04091) paper
as well as the [Baby-AGI](https://github.com/yoheinakajima/babyagi) project.

The core idea is to first come up with a multi-step plan, and then go through
that plan one item at a time. After accomplishing a particular task, you can
then revisit the plan and modify as appropriate.

This compares to a typical [ReAct](https://arxiv.org/abs/2210.03629) style agent
where you think one step at a time. The advantages of this "plan-and-execute"
style agent are:

1. Explicit long term planning (which even really strong LLMs can struggle with)
2. Ability to use smaller/weaker models for the execution step, only using
   larger/better models for the planning step


## Setup

First, we need to install the packages required.

```bash
npm install @langchain/langgraph @langchain/openai langchain
```


Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the
search tool we will use)


In [1]:
// process.env.OPENAI_API_KEY = "YOUR_API_KEY"
// process.env.TAVILY_API_KEY = "YOUR_API_KEY"


Optionally, we can set API key for LangSmith tracing, which will give us
best-in-class observability.


In [2]:
// process.env.LANGCHAIN_TRACING_V2 = "true"
// process.env.LANGCHAIN_API_KEY = "YOUR_API_KEY"
// process.env.LANGCHAIN_PROJECT = "YOUR_PROJECT_NAME"


## Define the State

Let's start by defining the state to track for this agent.

First, we will need to track the current plan. Let's represent that as a list of
strings.

Next, we should track previously executed steps. Let's represent that as a list
of tuples (these tuples will contain the step and then the result)

Finally, we need to have some state to represent the final response as well as
the original input.


In [1]:
import { Annotation } from "@langchain/langgraph";

const PlanExecuteState = Annotation.Root({
  input: Annotation<string>({
    reducer: (x, y) => y ?? x ?? "",
  }),
  plan: Annotation<string[]>({
    reducer: (x, y) => y ?? x ?? [],
  }),
  pastSteps: Annotation<[string, string][]>({
    reducer: (x, y) => x.concat(y),
  }),
  response: Annotation<string>({
    reducer: (x, y) => y ?? x,
  }),
})

## Define Tools

We will first define the tools we want to use. For this simple example, we will
use a built-in search tool via Tavily. However, it is really easy to create your
own tools - see documentation
[here](https://js.langchain.com/docs/modules/agents/tools/dynamic) on how to do
that.


In [2]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const tools = [new TavilySearchResults({ maxResults: 3 })];


## Define our Execution Agent

Now we will create the execution agent we want to use to execute tasks. Note
that for this example, we will be using the same execution agent for each task,
but this doesn't HAVE to be the case.


In [3]:
import { ChatOpenAI } from "@langchain/openai";
import { createReactAgent } from "@langchain/langgraph/prebuilt";

const agentExecutor = createReactAgent({
  llm: new ChatOpenAI({ model: "gpt-4o" }),
  tools: tools,
});


In [4]:
await agentExecutor.invoke({
  messages: ["user", "who is the winner of the us open"],
});


{
  messages: [
    'user',
    'who is the winner of the us open',
    AIMessage {
      "id": "chatcmpl-9yp7dHIgCdRgsJ1Ie58wdLpT5llsR",
      "content": "",
      "additional_kwargs": {
        "tool_calls": [
          {
            "id": "call_OgRCeSGYwBxBDnjUjmYmtGH6",
            "type": "function",
            "function": "[Object]"
          }
        ]
      },
      "response_metadata": {
        "tokenUsage": {
          "completionTokens": 24,
          "promptTokens": 85,
          "totalTokens": 109
        },
        "finish_reason": "tool_calls",
        "system_fingerprint": "fp_3aa7262c27"
      },
      "tool_calls": [
        {
          "name": "tavily_search_results_json",
          "args": {
            "input": "winner of US Open 2023"
          },
          "type": "tool_call",
          "id": "call_OgRCeSGYwBxBDnjUjmYmtGH6"
        }
      ],
      "invalid_tool_calls": [],
      "usage_metadata": {
        "input_tokens": 85,
        "output_tokens": 24,
    

## Planning Step

Let's now think about creating the planning step. This will use function calling
to create a plan.


In [5]:
import { z } from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";

const plan = zodToJsonSchema(
  z.object({
    steps: z
      .array(z.string())
      .describe("different steps to follow, should be in sorted order"),
  }),
);
const planFunction = {
  name: "plan",
  description: "This tool is used to plan the steps to follow",
  parameters: plan,
};

const planTool = {
  type: "function",
  function: planFunction,
};

In [6]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const plannerPrompt = ChatPromptTemplate.fromTemplate(
  `For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

{objective}`,
);

const model = new ChatOpenAI({
  modelName: "gpt-4-0125-preview",
}).withStructuredOutput(planFunction);

const planner = plannerPrompt.pipe(model);

In [7]:
await planner.invoke({
  objective: "what is the hometown of the current Australia open winner?",
});


{
  steps: [
    'Identify the current Australia Open winner.',
    'Find out the hometown of the identified winner.'
  ]
}


## Re-Plan Step

Now, let's create a step that re-does the plan based on the result of the
previous step.


In [8]:
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";

const response = zodToJsonSchema(
  z.object({
    response: z.string().describe("Response to user."),
  }),
);

const responseTool = {
  type: "function",
  function: {
    name: "response",
    description: "Response to user.",
    parameters: response,
  },
};

const replannerPrompt = ChatPromptTemplate.fromTemplate(
  `For the given objective, come up with a simple step by step plan. 
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps.
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{pastSteps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that and use the 'response' function.
Otherwise, fill out the plan.  
Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.`,
);

const parser = new JsonOutputToolsParser();
const replanner = replannerPrompt
  .pipe(
    new ChatOpenAI({ model: "gpt-4o" }).bindTools([
      planTool,
      responseTool,
    ]),
  )
  .pipe(parser);

## Create the Graph

We can now create the graph!


In [10]:
import { END, START, StateGraph } from "@langchain/langgraph";
import { RunnableConfig } from "@langchain/core/runnables";

async function executeStep(
  state: typeof PlanExecuteState.State,
  config?: RunnableConfig,
): Promise<Partial<typeof PlanExecuteState.State>> {
  const task = state.plan[0];
  const input = {
    messages: ["user", task],
  };
  const { messages } = await agentExecutor.invoke(input, config);

  return {
    pastSteps: [[task, messages[messages.length - 1].content.toString()]],
    plan: state.plan.slice(1),
  };
}

async function planStep(
  state: typeof PlanExecuteState.State,
): Promise<Partial<typeof PlanExecuteState.State>> {
  const plan = await planner.invoke({ objective: state.input });
  return { plan: plan.steps };
}

async function replanStep(
  state: typeof PlanExecuteState.State,
): Promise<Partial<typeof PlanExecuteState.State>> {
  const output = await replanner.invoke({
    input: state.input,
    plan: state.plan.join("\n"),
    pastSteps: state.pastSteps
      .map(([step, result]) => `${step}: ${result}`)
      .join("\n"),
  });
  const toolCall = output[0];

  if (toolCall.type == "response") {
    return { response: toolCall.args?.response };
  }

  return { plan: toolCall.args?.steps };
}

function shouldEnd(state: typeof PlanExecuteState.State) {
  return state.response ? "true" : "false";
}

const workflow = new StateGraph(PlanExecuteState)
  .addNode("planner", planStep)
  .addNode("agent", executeStep)
  .addNode("replan", replanStep)
  .addEdge(START, "planner")
  .addEdge("planner", "agent")
  .addEdge("agent", "replan")
  .addConditionalEdges("replan", shouldEnd, {
    true: END,
    false: "agent",
  });

// Finally, we compile it!
// This compiles it into a LangChain Runnable,
// meaning you can use it as you would any other runnable
const app = workflow.compile();

In [11]:
const config = { recursionLimit: 50 };
const inputs = {
  input: "what is the hometown of the 2024 Australian open winner?",
};

for await (const event of await app.stream(inputs, config)) {
  console.log(event);
}


{
  planner: {
    plan: [
      'Determine the winner of the 2024 Australian Open.',
      'Find out the hometown of the determined winner.',
      'Present the hometown as the final answer.'
    ]
  }
}
{
  agent: {
    plan: [
      'Find out the hometown of the determined winner.',
      'Present the hometown as the final answer.'
    ],
    pastSteps: [ [Array] ]
  }
}
{
  replan: {
    plan: [
      'Find out the hometown of Jannik Sinner.',
      'Find out the hometown of Aryna Sabalenka.',
      'Present the hometowns as the final answer.'
    ]
  }
}
{
  agent: {
    plan: [
      'Find out the hometown of Aryna Sabalenka.',
      'Present the hometowns as the final answer.'
    ],
    pastSteps: [ [Array] ]
  }
}
{
  replan: {
    plan: [
      'Find out the hometown of Aryna Sabalenka.',
      'Present the hometown of Aryna Sabalenka as the final answer.'
    ]
  }
}
{
  agent: {
    plan: [ 'Present the hometown of Aryna Sabalenka as the final answer.' ],
    pastSteps: [ [

> #### See the LangSmith trace [here](https://smith.langchain.com/public/2f43e8bb-56b8-46da-b059-c0f4c6e43cbb/r).
